In [39]:
import cv2
import numpy as np
from sklearn.svm import SVC
from sklearn.externals import joblib
from cv2 import HOGDescriptor
import LineDetector
from numpy.linalg import norm

class PathFinder: 
    
    #ranges for semafors
    red_low_range_low_h = 0
    red_low_range_low_s = 100
    red_low_range_low_v = 100
    red_low_range_high_h = 5
    red_low_range_high_s = 255
    red_low_range_high_v = 255
    red_high_range_low_h = 170
    red_high_range_low_s = 100
    red_high_range_low_v = 100
    red_high_range_high_h = 179
    red_high_range_high_s = 255
    red_high_range_high_v = 255
    
    def __init__(self,model_signs,model_semafors,printFlag,video_source=0,
                 resize=(640,480)):
        self.clfSigns = joblib.load(model_signs) #path to signs clf
        self.clfSem = joblib.load(model_semafors) #path to semafors clf        
        self.cap=cv2.VideoCapture(video_source)
        self.size = resize
        self.printFlag = printFlag #Flag for additional drawing

            
    def __hog2(self,image):
        winSize = (16,48)
        blockSize = (16,16)
        blockStride = (8,8)
        cellSize = (8,8)
        nbins = 9
        derivAperture = 1
        winSigma = 4.
        histogramNormType = 0
        L2HysThreshold = 2.0000000000000001e-01
        gammaCorrection = 100
        nlevels = 32
        hog = HOGDescriptor(winSize,blockSize,blockStride,cellSize,nbins,derivAperture,winSigma,
                        histogramNormType,L2HysThreshold,gammaCorrection,nlevels)
        winStride = (8,8)
        padding = (16,16)
        hist = hog.compute(image,winStride,padding)
        return np.reshape(hist, 4500)
    
    
    def __hog(self,img,step):
        winSize = (16,16)
        blockSize = (16,16)
        blockStride = (8,8)
        cellSize = (8,8)
        nbins = 9
        derivAperture = 1
        winSigma = 4.
        histogramNormType = 0
        L2HysThreshold = 2.0000000000000001e-01
        gammaCorrection = 0
        nlevels = 64
        hog = HOGDescriptor(winSize,blockSize,blockStride,cellSize,nbins,derivAperture,winSigma,
                            histogramNormType,L2HysThreshold,gammaCorrection,nlevels)
        winStride = (8,8)
        padding = (16,16)
        hist = hog.compute(img,winStride,padding)
        return np.reshape(hist, 4356)
    
    
    def StartRide(self):
        frame=self.cap.read()[1]
        self.line_detector = LineDetector.RoadControl(frame, ROI_height = 210, viz = True)
        self.line_detector.vec1 = [-3, -1, 70] 
        self.line_detector.vec2 = [3, -1, 70]       
        while(self.cap.isOpened() and len(frame)>0):
            frame = cv2.resize(frame,self.size, interpolation = cv2.INTER_CUBIC)
            predictSigns=self.__detectSigns(frame,self.printFlag)
            predictSemafors = self.__detectSemafors(frame,self.printFlag)         
            self.line_detector.img = frame
#            print(self.line_detector.poke())
            #HERE ADD YOUR MODULES
            cv2.imshow('frame2',frame)
            if cv2.waitKey(2) & 0xFF == ord("q"):
                break
            frame=self.cap.read()[1]
            
        self.cap.release()
        cv2.destroyAllWindows()
    
    def __detectSemafors(self,frame,printFlag):
        #noramalization
        frame2 = cv2.medianBlur(frame,3)      
        frame2 = cv2.GaussianBlur(frame2,(3,3),2) #important for finding circles!
        #masking
        hsv_image_1 = cv2.cvtColor(frame2, cv2.COLOR_BGR2HSV)
        red_mask_low = cv2.inRange(hsv_image_1,np.array((self.red_low_range_low_h,self.red_low_range_low_s,self.red_low_range_low_v)
                                                        , dtype = "uint8"),np.array((self.red_low_range_high_h, self.red_low_range_high_s,
                                                                                     self.red_low_range_high_v), dtype = "uint8"))
        red_mask_high = cv2.inRange(hsv_image_1,np.array((self.red_high_range_low_h,self.red_high_range_low_s,self.red_high_range_low_v),
                                                         dtype = "uint8"),np.array((self.red_high_range_high_h, self.red_high_range_high_s,
                                                                                    self.red_high_range_high_v), dtype = "uint8"))
        mask = cv2.addWeighted(red_mask_low,1.0, red_mask_high,1.0, 0.0)        
        #additional drawing
        if(printFlag):
            cv2.imshow('Mask_Semafors',mask)
        #find a circles from mask    
        circles = cv2.HoughCircles(mask,cv2.HOUGH_GRADIENT,1,70,param1=70,param2=10,minRadius=5,maxRadius=20)
        if np.all(circles) != None:
            circles = np.uint16(np.around(circles))
            for i in circles[0,:]:
                if True:                    
                    x1=i[0]-i[2]*2
                    y1=i[1]-i[2]*2
                    x2=i[0]+i[2]*2
                    y2=i[1]+i[2]*7
                    if y2>self.size[1]-1:
                        y2=self.size[1]-1
                    if x2>self.size[0]-1:
                        x2=self.size[0]-1
                    if(x1 > 0 and x2 >0 and y1 > 0 and y2 > 0):                     
                        cut_frame = frame2[y1:y2,x1:x2]
                        cut_frame = cv2.resize(cut_frame,(16, 48), interpolation = cv2.INTER_CUBIC)
                        pred = self.clfSem.predict_proba([self.__hog2(cut_frame)])[0][1]
                        if(pred>0.6):    
                            if(printFlag):
                                cv2.circle(frame,(i[0],i[1]),i[2],(0,255,255),2)
                                frame = cv2.rectangle(frame,(x1,y1),(x2,y2),(0,0,255),2)
                            return True
                        return False
                    
    
    def __detectSigns(self,frame,printFlag):
        #noramalization
        frame2 = cv2.medianBlur(frame,3)      
        frame2 = cv2.GaussianBlur(frame2,(3,3),2) #important for finding circles!
        #masking
        hsv_image_1 = cv2.cvtColor(frame2, cv2.COLOR_BGR2HSV)
        low_blue = (95,100,100)
        hight_blue = (120,255,255)
        mask = cv2.inRange(hsv_image_1, low_blue, hight_blue)  
        if(printFlag):
            cv2.imshow('Mask_Signs',mask)  
        circles = cv2.HoughCircles(mask,cv2.HOUGH_GRADIENT,1,50,param1=70,param2=10,minRadius=10,maxRadius=40) #let's change params
        pred=[]
        if np.all(circles) != None:
            circles = np.round(circles[0, :]).astype("int")
            for (x, y, r) in circles:
                if((x-r) > 0 and (x+r) >0 and (y-r)>0 and (y+r)>0):
                    cut_frame = frame2[y-r:y+r,x-r:x+r]                    
                    cut_frame = cv2.resize(cut_frame,(64, 64), interpolation = cv2.INTER_CUBIC)
                    hog=self.__hog(cut_frame,8)
                    k = self.clfSigns.predict_proba([hog])[0]
                    maximum = max(k)
                    #print(max(k))
                    index = np.where(k == maximum)[0][0]
                    #print(index[0][0])
                    if(maximum>0.9):
                        pred.append(index)
                        if(printFlag):
                            frame = cv2.rectangle(frame,(x-r,y-r),(x+r,y+r),(0,0,255),2)            
                            cv2.imshow('Sign_cut:'+str(index),cut_frame)
        return pred
    
    
    

In [40]:
One = PathFinder('model.pkl','semafor/model_semafors.pkl',True,'signs/signs_test.mp4')
One.StartRide()